In [1]:
import numpy as np
import tensorflow as tf
import ampligraph
import pandas as pd
import sklearn as sk
from sklearn.model_selection import train_test_split
from ampligraph.latent_features import save_model, restore_model
from tqdm import tqdm
import imblearn


print(ampligraph.__version__)
print(tf.__version__)
print(sk.__version__)
print(np.__version__)

1.4.0
1.15.0
0.24.2
1.20.3
0.8.0


In [2]:
import ampligraph
bankTriples = pd.read_csv("BankTriples.csv")

telcoTriples = pd.read_csv("TelcoTriples.csv")
    
bankData = pd.read_csv("bank_prepped.csv")

telcoData = pd.read_csv("Telco_prepped.csv")

telcoTriples.head()

,Unnamed: 0,subject,predicate,object
0,0,8668-KNZTI,hasZipCode,95834
1,1,8668-KNZTI,hasChurned,False
2,2,8668-KNZTI,totalCharges,2790.65
3,3,8668-KNZTI,hasMonthlyCharges,53.75
4,4,8668-KNZTI,usesPaymentMethod,Electronic check


In [3]:
bankData = bankData.sample(frac=1)

bankData.head()



,Unnamed: 0,CustomerId,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
3283,3283,15807163,537,France,Female,38,10,0.00,1,False,False,52337.97,True
1729,1729,15596647,768,France,Male,54,8,69712.74,1,True,True,69381.05,False
3784,3784,15605072,638,France,Female,43,3,145860.98,1,True,True,142763.51,True
3063,3063,15691004,407,Spain,Male,37,1,0.00,1,True,True,49161.12,True
868,868,15723856,602,France,Female,29,3,88814.40,2,True,True,62487.97,False


In [4]:
bank_x = bankData.loc[:, bankData.columns != 'Exited']

bank_y = bankData["Exited"]

telco_x = telcoData.loc[:, telcoData.columns != 'Churn Value']

telco_y = telcoData["Churn Value"]

In [5]:
X_bank_train, X_bank_test, y_bank_train, y_bank_test = train_test_split(
    bank_x, bank_y, test_size=0.20, random_state=145)

X_telco_train, X_telco_test, y_telco_train, y_telco_test = train_test_split(
    telco_x, telco_y, test_size=0.20, random_state=145)

In [6]:
#subsetting telco data

ids = X_telco_test["CustomerID"]
    
idList = list(ids)
    
testTriplesTelco = telcoTriples[(telcoTriples["subject"].isin(idList)) & (telcoTriples["predicate"] == "hasChurned")]
    

trainTriplesTelco = telcoTriples.merge(testTriplesTelco,how ="left",indicator=True)

trainTriplesTelco = trainTriplesTelco[trainTriplesTelco['_merge'] == "left_only"]


del trainTriplesTelco["Unnamed: 0"]
del trainTriplesTelco["_merge"]
del testTriplesTelco["Unnamed: 0"]


telcoFilters = pd.concat([trainTriplesTelco, testTriplesTelco])


#Subsetting the Bank Triples 
ids = X_bank_test["CustomerId"]
    
idList = list(ids)
    
testTriplesBank = bankTriples[(bankTriples["subject"].isin(idList)) & (bankTriples["predicate"] == "hasChurned")]
    

trainTriplesBank = bankTriples.merge(testTriplesBank,how ="left",indicator=True)

trainTriplesBank = trainTriplesBank[trainTriplesBank['_merge'] == "left_only"]





del trainTriplesBank["Unnamed: 0"]
del trainTriplesBank["_merge"]
del testTriplesBank["Unnamed: 0"]


trainTriplesBank["subject"] = trainTriplesBank["subject"].astype(str)






bankFilters = pd.concat([trainTriplesBank,testTriplesBank])



      subject          predicate            object
0  8668-KNZTI         hasZipCode             95834
1  8668-KNZTI         hasChurned             False
2  8668-KNZTI       totalCharges           2790.65
3  8668-KNZTI  hasMonthlyCharges             53.75
4  8668-KNZTI  usesPaymentMethod  Electronic check
        subject   predicate object
93   9575-IWCAZ  hasChurned  False
162  1552-CZCLL  hasChurned  False
254  7036-ZZKBD  hasChurned  False
277  9314-QDMDW  hasChurned  False
622  0793-TWELN  hasChurned  False
85226
748
######
      subject   predicate object
96   15596914  hasChurned  False
129  15585041  hasChurned  False
206  15576723  hasChurned  False
283  15736371  hasChurned  False
294  15619932  hasChurned  False
    subject         predicate  object
0  15692761  hasCreditScoreOf     718
1  15692761         hasGender    Male
2  15692761             isAge      36
3  15692761         inCountry  France
4  15692761        hasBalance     0.0
43999


,subject,predicate,object
0,15692761,hasCreditScoreOf,718
1,15692761,hasGender,Male
2,15692761,isAge,36
3,15692761,inCountry,France
4,15692761,hasBalance,0.0


In [7]:
trainTriplesBank[(trainTriplesBank.predicate=="hasChurned")]

,subject,predicate,object
8,15692761,hasChurned,False
19,15700046,hasChurned,False
30,15709324,hasChurned,False
41,15745452,hasChurned,False
52,15599081,hasChurned,False
...,...,...,...
44756,15666295,hasChurned,True
44767,15672754,hasChurned,True
44778,15768163,hasChurned,True
44800,15584532,hasChurned,True


In [8]:
trainTriplesTelco[(trainTriplesTelco.predicate=="hasChurned")]

,subject,predicate,object
1,8668-KNZTI,hasChurned,False
24,1099-BTKWT,hasChurned,False
47,1536-YHDOE,hasChurned,False
70,7048-GXDAY,hasChurned,False
116,1027-LKKQQ,hasChurned,False
...,...,...,...
85837,1122-JWTJW,hasChurned,True
85860,1699-HPSBG,hasChurned,True
85883,8775-CEBBJ,hasChurned,True
85929,0639-TSIQW,hasChurned,True


# TransE model

In [9]:
#TransE model training
from ampligraph.latent_features import TransE


transEModelBank = TransE(batches_count=20,
                    seed=1525,
                    epochs=500,
                    k=45,
                    eta=20,
                    optimizer='adam',
                    optimizer_params={'lr':1e-3},
                    loss='pairwise',
                    regularizer='LP', 
                    regularizer_params={'p':2, 'lambda':1e-5},
                    verbose=True)



In [10]:
transEModelBankHistory = transEModelBank.fit(trainTriplesBank.to_numpy(), early_stopping = False,
              early_stopping_params = 
                   {
                  'x_filter': bankFilters     
              })

Average TransE Loss:   0.049648: 100%|██████████| 500/500 [01:23<00:00,  5.96epoch/s]


In [11]:
save_model(transEModelBank, './models/transEModelBank.pkl')

In [12]:
transEModelTelco = TransE(batches_count=20,
                    seed=42,
                    epochs=500,
                    k=65,
                    eta=20,
                    optimizer='adam',
                    optimizer_params={'lr':1e-3},
                    loss='pairwise',
                    regularizer='LP', 
                    regularizer_params={'p':2, 'lambda':1e-5},
                    verbose=True)

In [13]:
transEModelTelco.fit(trainTriplesTelco.to_numpy(),
              early_stopping_params = 
                   {
                  'x_filter': telcoFilters
              })

Average TransE Loss:   0.052764: 100%|██████████| 500/500 [02:41<00:00,  3.10epoch/s]


In [14]:
save_model(transEModelTelco, './models/transEModelTelco.pkl')

# DistMult

In [15]:
#DistMult
from ampligraph.latent_features import DistMult


distmultModelBank = DistMult(batches_count=20,
                      seed=57,
                      epochs=500, 
                      k=12,
                      eta=20,
                     optimizer='adam',
                     optimizer_params={'lr':1e-3},
                      loss='pairwise',
                    regularizer='LP', 
                    regularizer_params={'p':1, 'lambda':1e-5},
                     verbose=True)



In [16]:
distmultModelBank.fit(trainTriplesBank.to_numpy(),
              early_stopping_params = 
                   {
                  'x_filter': bankFilters
              })

Average DistMult Loss:   0.058712: 100%|██████████| 500/500 [00:57<00:00,  8.66epoch/s]


In [17]:
save_model(distmultModelBank, './models/distmultModelBank.pkl')

In [18]:
distmultModelTelco = DistMult(batches_count=20,
                      seed=475,
                      epochs=500, 
                      k=16,
                      eta=20,
                     optimizer='adam',
                    optimizer_params={'lr':1e-3},
                      loss='pairwise',
                    regularizer='LP', 
                    regularizer_params={'p':1, 'lambda':1e-5},
                     verbose=True)

In [19]:
distmultModelTelco.fit(trainTriplesTelco.to_numpy(),
              early_stopping_params = 
                   {
                  'x_filter': telcoFilters   
              })


Average DistMult Loss:   0.059675: 100%|██████████| 500/500 [01:33<00:00,  5.37epoch/s]


In [20]:
save_model(distmultModelTelco, './models/distmultModelTelco.pkl')

# Complex model training

In [21]:
#Complex model
from ampligraph.latent_features import ComplEx

In [22]:
complexModelBank= ComplEx(batches_count=20,
                          seed=34,
                          epochs=500, 
                          k=5,
                          eta=20,
                         optimizer='adam',
                        optimizer_params={'lr':1e-3},
                          loss='pairwise',
                          regularizer='LP', 
                regularizer_params={'p':3, 'lambda':1e-5},
                         verbose=True)

In [23]:
complexModelBank.fit(trainTriplesBank.to_numpy(),
              early_stopping_params = 
                   {
                  'x_filter': bankFilters
              })

Average ComplEx Loss:   0.066507: 100%|██████████| 500/500 [01:26<00:00,  5.78epoch/s]


In [24]:
save_model(complexModelBank, './models/complexModelBank.pkl')

In [25]:
complexModelTelco= ComplEx(batches_count=20,
                           seed=85,
                          epochs=500, 
                          k=10,
                          eta=20,
                         optimizer='adam',
                           optimizer_params={'lr':1e-3},
                          loss='pairwise',
                           regularizer='LP', 
                    regularizer_params={'p':3, 'lambda':1e-5},
                         verbose=True)

In [26]:
complexModelTelco.fit(trainTriplesTelco.to_numpy(),
              early_stopping_params = 
                   {
                  'x_filter': telcoFilters 
              })

Average ComplEx Loss:   0.055131: 100%|██████████| 500/500 [02:30<00:00,  3.33epoch/s]


In [27]:
save_model(complexModelTelco, './models/complexModelTelco.pkl')

In [28]:
testTriplesTelco.head()

,subject,predicate,object
93,9575-IWCAZ,hasChurned,False
162,1552-CZCLL,hasChurned,False
254,7036-ZZKBD,hasChurned,False
277,9314-QDMDW,hasChurned,False
622,0793-TWELN,hasChurned,False


# Function for making predictions 

In [29]:

def makePredictions(model, testData):
    preds = []
    for k in tqdm(range(0,len(testData))):
                   
        row =   testData.iloc[k]        
                   
        pred1 = model.predict([row["subject"],row["predicate"],"True"])[0]
        pred2 = model.predict([row["subject"],row["predicate"],"False"])[0]
        
        if pred1 > pred2:
            preds.append("True")
        else:
             preds.append("False")
            
    return preds
    

# Testing the models
Telco First

In [30]:
resultsTransETelco = makePredictions(transEModelTelco, testTriplesTelco)


100%|██████████| 748/748 [05:07<00:00,  2.43it/s]


In [31]:
resultsDistmultTelco = makePredictions(distmultModelTelco, testTriplesTelco)

100%|██████████| 748/748 [02:00<00:00,  6.20it/s]


In [32]:
resultsComplexTelco = makePredictions(complexModelTelco, testTriplesTelco)

100%|██████████| 748/748 [03:12<00:00,  3.89it/s]


In [33]:
trueResultsTelco = testTriplesTelco["object"]

telcoDict = {"transe":resultsTransETelco,"dis":resultsDistmultTelco, "complex":resultsComplexTelco, "true":trueResultsTelco}

telcoResults = pd.DataFrame(telcoDict)

telcoResults.to_csv("results/telcoResults.csv")

Now Bank data

In [34]:
resultsTransEBank = makePredictions(transEModelBank, testTriplesBank)

100%|██████████| 815/815 [03:45<00:00,  3.62it/s]


In [35]:
resultsDistmultBank = makePredictions(distmultModelBank, testTriplesBank)

100%|██████████| 815/815 [01:45<00:00,  7.70it/s]


In [36]:
resultsComplexBank = makePredictions(complexModelBank, testTriplesBank)

100%|██████████| 815/815 [02:31<00:00,  5.37it/s]


In [37]:
trueResultsBank = testTriplesBank["object"]

bankDict = {"transe":resultsTransEBank,"dis":resultsDistmultBank, "complex":resultsComplexBank, "true":trueResultsBank}

bankResults = pd.DataFrame(bankDict)

bankResults.to_csv("results/bankResults.csv")

Saving data so I can use the same split as for the baseline approaches

In [38]:
X_bank_train.to_csv("splitData/bank/X_bank_train.csv")
X_bank_test.to_csv("splitData/bank/X_bank_test.csv")

y_bank_train.to_csv("splitData/bank/y_bank_train.csv")
y_bank_test.to_csv("splitData/bank/y_bank_test.csv")

In [39]:
X_telco_train.to_csv("splitData/telco/X_telco_train.csv")
X_telco_test.to_csv("splitData/telco/X_telco_test.csv")

y_telco_train.to_csv("splitData/telco/y_telco_train.csv")
y_telco_test.to_csv("splitData/telco/y_telco_test.csv")

In [2]:
from ampligraph.utils import restore_model


In [3]:
#Loading the models again to fetch paramters
#TransE
transEModelBank = restore_model(model_name_path = './models/transEModelBank.pkl')
transEModelTelco = restore_model(model_name_path = './models/transEModelTelco.pkl')

#DisMult
distmultModelBank = restore_model(model_name_path = './models/distmultModelBank.pkl')
distmultModelTelco = restore_model(model_name_path =  './models/distmultModelTelco.pkl')

#complEx
complexModelBank = restore_model(model_name_path = './models/complexModelBank.pkl')
complexModelTelco = restore_model(model_name_path = './models/complexModelTelco.pkl')

In [4]:
transEModelBank.get_hyperparameter_dict()

{'k': 45,
 'eta': 20,
 'epochs': 500,
 'batches_count': 20,
 'seed': 1525,
 'embedding_model_params': {'norm': 1,
  'normalize_ent_emb': False,
  'negative_corruption_entities': 'all',
  'corrupt_sides': ['s,o']},
 'optimizer': 'adam',
 'optimizer_params': {'lr': 0.001},
 'loss': 'pairwise',
 'loss_params': {},
 'regularizer': 'LP',
 'regularizer_params': {'p': 2, 'lambda': 1e-05},
 'initializer': 'xavier',
 'initializer_params': {'uniform': False},
 'verbose': True}

In [5]:
transEModelTelco.get_hyperparameter_dict()

{'k': 65,
 'eta': 20,
 'epochs': 500,
 'batches_count': 20,
 'seed': 42,
 'embedding_model_params': {'norm': 1,
  'normalize_ent_emb': False,
  'negative_corruption_entities': 'all',
  'corrupt_sides': ['s,o']},
 'optimizer': 'adam',
 'optimizer_params': {'lr': 0.001},
 'loss': 'pairwise',
 'loss_params': {},
 'regularizer': 'LP',
 'regularizer_params': {'p': 2, 'lambda': 1e-05},
 'initializer': 'xavier',
 'initializer_params': {'uniform': False},
 'verbose': True}

In [6]:
distmultModelBank.get_hyperparameter_dict()

{'k': 12,
 'eta': 20,
 'epochs': 500,
 'batches_count': 20,
 'seed': 57,
 'embedding_model_params': {'normalize_ent_emb': False,
  'negative_corruption_entities': 'all',
  'corrupt_sides': ['s,o']},
 'optimizer': 'adam',
 'optimizer_params': {'lr': 0.001},
 'loss': 'pairwise',
 'loss_params': {},
 'regularizer': 'LP',
 'regularizer_params': {'p': 1, 'lambda': 1e-05},
 'initializer': 'xavier',
 'initializer_params': {'uniform': False},
 'verbose': True}

In [10]:
distmultModelTelco.get_hyperparameter_dict()

{'k': 16,
 'eta': 20,
 'epochs': 500,
 'batches_count': 20,
 'seed': 475,
 'embedding_model_params': {'normalize_ent_emb': False,
  'negative_corruption_entities': 'all',
  'corrupt_sides': ['s,o']},
 'optimizer': 'adam',
 'optimizer_params': {'lr': 0.001},
 'loss': 'pairwise',
 'loss_params': {},
 'regularizer': 'LP',
 'regularizer_params': {'p': 1, 'lambda': 1e-05},
 'initializer': 'xavier',
 'initializer_params': {'uniform': False},
 'verbose': True}

In [8]:
complexModelBank.get_hyperparameter_dict()

{'k': 5,
 'eta': 20,
 'epochs': 500,
 'batches_count': 20,
 'seed': 34,
 'embedding_model_params': {'negative_corruption_entities': 'all',
  'corrupt_sides': ['s,o']},
 'optimizer': 'adam',
 'optimizer_params': {'lr': 0.001},
 'loss': 'pairwise',
 'loss_params': {},
 'regularizer': 'LP',
 'regularizer_params': {'p': 3, 'lambda': 1e-05},
 'initializer': 'xavier',
 'initializer_params': {'uniform': False},
 'verbose': True}

In [9]:
complexModelTelco.get_hyperparameter_dict()

{'k': 10,
 'eta': 20,
 'epochs': 500,
 'batches_count': 20,
 'seed': 85,
 'embedding_model_params': {'negative_corruption_entities': 'all',
  'corrupt_sides': ['s,o']},
 'optimizer': 'adam',
 'optimizer_params': {'lr': 0.001},
 'loss': 'pairwise',
 'loss_params': {},
 'regularizer': 'LP',
 'regularizer_params': {'p': 3, 'lambda': 1e-05},
 'initializer': 'xavier',
 'initializer_params': {'uniform': False},
 'verbose': True}